#### testing out the encoder replacement in this section and see if it works or not

##### testing by adding the horizontal and vertical sections together 

##### the inference speed and model size decrease

In [1]:
import os 
import cv2 
import argparse 
import torch 
import torchvision.transforms as T

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from repnet import utils, plots
from repnet.model import RepNet

import event_tools.tools as tl 
import visualization as vz 
import event_tools.e2v as e2v 


In [2]:
event_class = "class2"
# event_user = "user02_fluorescent"
event_user = "user02_fluorescent_led"
# event_user = "user02_lab"
# event_user = "user02_led"
# event_user = "user02_natural"

event_csv_file_name = f"./event_csv/split_data/{event_class}/{event_user}.csv"

count_data = {
    "class": event_class,
    "condition": event_user,
    "count": 0,
}

# output count info 
out_csv_file = "encoder_replaced_output.csv"

if os.path.exists(out_csv_file):
    try: 
        out_df = pd.read_csv(out_csv_file)
    except: 
        out_df = pd.DataFrame([count_data])
else: 
    with open(out_csv_file, "w") as f:
        f.write("class,condition,count\n")
    out_df = pd.read_csv(out_csv_file)


In [3]:
# out_df = pd.concat([out_df, pd.DataFrame(columns=["class", "condition", "count"])])

out_df

,class,condition,count
0,class4,user02_led,3
1,class4,user02_led,3
2,class3,user02_led,11
3,class3,user02_lab,11
4,class3,user02_natural,3


In [4]:
# video_path = './'
events = tl.load_event_data(
    # './event_csv/split_data/class7/user02_lab.csv')  # Replace with your actual file path
    event_csv_file_name) # Replace with your actual file path

In [5]:
# repnet model variables 
weights = './pytorch_weights.pth'
device = 'cuda'
strides = [1, 2, 3, 4 , 8]


OUT_VISUALIZATIONS_DIR = './visualization/'


In [13]:
# spiking variables

width=128
height=128

# horizontal_member_potential = torch.zeros(width).to(device)
# vertical_member_potential = torch.zeros(width).to(device) 

horizontal_member_potential = torch.zeros(width).reshape((1, -1)).to(device)        
vertical_member_potential = torch.zeros(width).reshape((1, -1)).to(device)


# horizontal_neurons = np.zeros(width).reshape((1, -1))
# vertical_neurons = np.zeros(height).reshape((1, -1))

# horizontal_member_potential = torch.zeros((6, width), device=device) 
# vertical_member_potential = torch.zeros((64, width), device=device) 

# horizontal_neurons = torch.zeros((64, width), device=device).reshape((1, -1))
# vertical_neurons = torch.zeros((64, height), device=device).reshape((1, -1))



decay = .009 # 0.009
batch_size = 1 #  64 #  
random_wt_2d = lambda _row, _col: torch.ones((_row, _col), device=device) * 0.5 # np.round(np.random.rand(_row, _col), 3)
weight_input_horizontal_hidden = random_wt_2d(batch_size, height) # np.ones((1, height)) * weight_scale_1
weight_input_horizontal_hidden_1 = random_wt_2d(height, height//2) # np.ones((height, height//2)) * weight_scale_1

# not considering the height and width for the image as they are symmetrical 
weight_input_vertical_hidden = random_wt_2d(batch_size, height) # np.ones((1, height)) * weight_scale_1
weight_input_vertical_hidden_1 = random_wt_2d(height, height//2) # np.ones((height, height//2)) * weight_scale_1



peak = 255 
spike = 1 # 255 
threshold = 0.8 

fps = 120
frame_per_second  = 1/fps

In [14]:
# e2v.save_event_as_video(events,name="event.mp4", frame_per_second=frame_per_second, decay=0.009)

In [15]:
# read frames and apply preprocessing 
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((112, 112)),
    T.ToTensor(),
    T.Normalize(mean=0.5, std=0.5),      
])

raw_frames, frames = [], [] 

for idx, frame_image in enumerate(
    e2v.get_event_frame(events, frame_per_second = frame_per_second, 
        decay= decay)):
    frames.append(torch.from_numpy(frame_image.copy()).float().to(device))
    frame_image = frame_image #.T
    cv2.imshow('raw' , frame_image)
    
    frame_image = np.clip(frame_image * 255, 0, 255).astype(np.uint8)
    frame_image = cv2.cvtColor(frame_image, cv2.COLOR_GRAY2BGR)
    # cv2.imshow('test', cv2.resize(frame_image, [frame_image.shape[0] * 10, frame_image.shape[0] * 10] )) 
    
    # if(cv2.waitKey(1) == ord('q')):
       
    #     break 
    raw_frames.append(frame_image)
    # frame = transform(frame_image)
    # frames.append(frame)
cv2.destroyAllWindows()

In [16]:
len(raw_frames), len(frames), type(frames)

(642, 642, list)

In [17]:
# Load model
model = RepNet()
state_dict = torch.load(weights)
model.load_state_dict(state_dict)

if hasattr(model, 'encoder'):
    del model.encoder

model.eval()
model.to(device)

Using cache found in /home/rlwagun/.cache/torch/hub/huggingface_pytorch-image-models_main
/tmp/ipykernel_2486680/1317078289.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

RepNet(
  (temporal_conv): Sequential(
    (0): Conv3d(1024, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(3, 1, 1), dilation=(3, 1, 1))
    (1): BatchNorm3d(512, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AdaptiveMaxPool3d(output_size=(None, 1, 1))
    (4): Flatten(start_dim=2, end_dim=4)
  )
  (tsm_conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (period_length_head): Sequential(
    (0): TranformerLayer(
      (input_projection): Linear(in_features=2048, out_features=512, bias=True)
      (transformer_layer): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=51

In [18]:
import torch
import psutil

# GPU Memory Usage
gpu_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MB
print(f"GPU Memory Usage: {gpu_memory:.2f} MB")

# CPU RAM Usage
ram_usage = psutil.Process().memory_info().rss / (1024 * 1024)  # Convert to MB
print(f"CPU RAM Usage: {ram_usage:.2f} MB")

def get_model_size(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())  # Model parameters
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())  # Model buffers (like BatchNorm stats)
    total_size_mb = (param_size + buffer_size) / (1024 * 1024)  # Convert to MB
    return total_size_mb

model_size = get_model_size(model)
print(f"Model Size: {model_size:.2f} MB")

GPU Memory Usage: 196.45 MB
CPU RAM Usage: 1109.16 MB
Model Size: 78.13 MB


In [19]:
# %%timeit
# frames = torch.stack(frames)
len(frames)


spikes_collection = []
# Process frames sequentially while optimizing computations
# for idx in range(frames.shape[0]):  # Stride-based loop
for idx in range(len(frames)):  # Stride-based loop
    frame_image = frames[idx]

    # Update potentials sequentially
    horizontal_member_potential.mul_(decay).add_(weight_input_horizontal_hidden @ frame_image)
    vertical_member_potential.mul_(decay).add_(weight_input_vertical_hidden @ frame_image.T)

    # Generate spikes
    spike_mask_h = horizontal_member_potential > (spike * threshold)
    spike_mask_v = vertical_member_potential > (spike * threshold)

    horizontal_neurons = spike_mask_h.float()
    vertical_neurons = spike_mask_v.float()

    horizontal_member_potential.masked_fill_(spike_mask_h, 0).relu_()
    vertical_member_potential.masked_fill_(spike_mask_v, 0).relu_()

    # Collect spikes
    spikes_collection.append(torch.cat([horizontal_neurons, vertical_neurons], dim=-1))


In [20]:
# %%timeit

import torch

# Initialize potentials
horizontal_member_potential = torch.zeros((1, width), device=device)
vertical_member_potential = torch.zeros((1, width), device=device)

best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = None, None, None, None, None, None

for stride in strides:
    # apply stride 
    stride_frames = spikes_collection[::stride]
    stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
    if len(stride_frames) < 64:
        continue # Skip this stride if there are not enough frames 
        
    raw_period_length, raw_periodicity_score, embeddings = [], [], []

    # iterate in steps of 64 frames 
    for start_idx in range(0, len(stride_frames) - 63, 64):
        window_frames = stride_frames[start_idx:start_idx + 64] 
        # Convert to tensor and reshape
        spikes_0 = torch.stack(window_frames).view(-1, 64, 128 * 2).to(device)
    
        with torch.no_grad():
            batch_period_length, batch_periodicity, batch_embeddings = model(spikes_0)

                # Store results
            raw_period_length.append(batch_period_length[0].cpu())
            raw_periodicity_score.append(batch_periodicity[0].cpu())
            embeddings.append(batch_embeddings[0].cpu())

    # Post-process results
    # if raw_period_length and raw_periodicity_score:
    raw_period_length, raw_periodicity_score, embeddings = map(torch.cat, [raw_period_length, raw_periodicity_score, embeddings])
    confidence, period_length, period_count, periodicity_score = model.get_counts(raw_period_length, raw_periodicity_score, stride)

    if best_confidence is None or confidence > best_confidence:
        best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = stride, confidence, period_length, period_count, periodicity_score, embeddings

# Final check
if best_stride is None:
    raise RuntimeError('Stride values too large; no 64-frame chunk could be sampled. Try different --strides values.')

print(f'Predicted period length: {best_period_length/fps:.1f} sec (~{int(best_period_length)} frames) with confidence {best_confidence:.2f}, using a stride of {best_stride}.')


Predicted period length: 0.4 sec (~53 frames) with confidence 0.90, using a stride of 4.


In [35]:
# %%timeit

# # works but really slow 

# import torch

# # Initialize potentials
# horizontal_member_potential = torch.zeros((1, width), device=device)
# vertical_member_potential = torch.zeros((1, width), device=device)

# best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = None, None, None, None, None, None

# spikes_collection = []
# # Process frames sequentially while optimizing computations
# # for idx in range(frames.shape[0]):  # Stride-based loop
# for idx in range(len(frames)):  # Stride-based loop
#     frame_image = frames[idx]

#     # Update potentials sequentially
#     horizontal_member_potential.mul_(decay).add_(weight_input_horizontal_hidden @ frame_image)
#     vertical_member_potential.mul_(decay).add_(weight_input_vertical_hidden @ frame_image.T)

#     # Generate spikes
#     spike_mask_h = horizontal_member_potential > (spike * threshold)
#     spike_mask_v = vertical_member_potential > (spike * threshold)

#     horizontal_neurons = spike_mask_h.float()
#     vertical_neurons = spike_mask_v.float()

#     horizontal_member_potential.masked_fill_(spike_mask_h, 0).relu_()
#     vertical_member_potential.masked_fill_(spike_mask_v, 0).relu_()

#     # Collect spikes
#     spikes_collection.append(torch.cat([horizontal_neurons, vertical_neurons], dim=-1))


# for stride in strides:
#     # apply stride 
#     stride_frames = spikes_collection[::stride]
#     stride_frames = stride_frames[:(len(stride_frames) // 64) * 64]
#     if len(stride_frames) < 64:
#         continue # Skip this stride if there are not enough frames 
        
#     raw_period_length, raw_periodicity_score, embeddings = [], [], []

#     # # Process batch inference every 64 frames
#     # if len(stride_frames) == 64:
#     for i in range(0, len(stride_frames) - 64  +1 , 64):
#         batch_frames = stride_frames[i:i + 64]
#         batch_frames = torch.stack(batch_frames).view(-1, 64, 128 * 2).to(device)

#         # Forward pass
#         with torch.no_grad():
#             batch_period_length, batch_periodicity, batch_embeddings = model(batch_frames)

#         # Store results
#         raw_period_length.append(batch_period_length[0].cpu())
#         raw_periodicity_score.append(batch_periodicity[0].cpu())
#         embeddings.append(batch_embeddings[0].cpu())
#     # with torch.no_grad():
#     #     spikes_0 = torch.stack(stride_frames).view(-1, 64, 128 * 2).to(device)
#     #     batch_period_length, batch_periodicity, batch_embeddings = model(spikes_0)

#     #     # Store results
#     #     raw_period_length.append(batch_period_length[0].cpu())
#     #     raw_periodicity_score.append(batch_periodicity[0].cpu())
#     #     embeddings.append(batch_embeddings[0].cpu())

#     # Post-process results
#     if raw_period_length and raw_periodicity_score:
#         raw_period_length, raw_periodicity_score, embeddings = map(torch.cat, [raw_period_length, raw_periodicity_score, embeddings])
#         confidence, period_length, period_count, periodicity_score = model.get_counts(raw_period_length, raw_periodicity_score, stride)

#         if best_confidence is None or confidence > best_confidence:
#             best_stride, best_confidence, best_period_length, best_period_count, best_periodicity_score, best_embeddings = stride, confidence, period_length, period_count, periodicity_score, embeddings

# # Final check
# if best_stride is None:
#     raise RuntimeError('Stride values too large; no 64-frame chunk could be sampled. Try different --strides values.')

# print(f'Predicted period length: {best_period_length/fps:.1f} sec (~{int(best_period_length)} frames) with confidence {best_confidence:.2f}, using a stride of {best_stride}.')


In [21]:
# Generate video with counts
rep_frames = plots.plot_repetitions(raw_frames[:len(best_period_count)], best_period_count.tolist(), best_periodicity_score.tolist() if not True else None)
video = cv2.VideoWriter(os.path.join(OUT_VISUALIZATIONS_DIR, 'repetitions.mp4'), cv2.VideoWriter_fourcc(*'mp4v'), fps, rep_frames[0].shape[:2][::-1])
for frame in rep_frames:
    video.write(frame)
video.release()

print('Done')

Done


In [22]:
len(best_period_count)

512

In [23]:
len(raw_frames), len(best_period_count)

(642, 512)

In [24]:
# best_period_count.tolist()

In [25]:
best_period_count.tolist()[-1]
round(best_period_count.tolist()[-1])

10

In [26]:
best_period_count

tensor([0.0208, 0.0417, 0.0625, 0.0833, 0.1042, 0.1250, 0.1458, 0.1667, 0.1875,
        0.2083, 0.2292, 0.2500, 0.2708, 0.2917, 0.3125, 0.3333, 0.3542, 0.3750,
        0.3958, 0.4167, 0.4375, 0.4583, 0.4792, 0.5000, 0.5208, 0.5417, 0.5625,
        0.5833, 0.6042, 0.6250, 0.6458, 0.6667, 0.6875, 0.7083, 0.7292, 0.7500,
        0.7708, 0.7917, 0.8125, 0.8333, 0.8526, 0.8718, 0.8910, 0.9103, 0.9295,
        0.9487, 0.9679, 0.9872, 1.0064, 1.0256, 1.0449, 1.0641, 1.0833, 1.1026,
        1.1218, 1.1410, 1.1603, 1.1795, 1.1987, 1.2179, 1.2372, 1.2564, 1.2756,
        1.2949, 1.3141, 1.3333, 1.3526, 1.3718, 1.3910, 1.4103, 1.4295, 1.4487,
        1.4679, 1.4872, 1.5064, 1.5256, 1.5449, 1.5641, 1.5833, 1.6026, 1.6218,
        1.6410, 1.6603, 1.6795, 1.6987, 1.7179, 1.7372, 1.7564, 1.7756, 1.7949,
        1.8141, 1.8333, 1.8526, 1.8718, 1.8910, 1.9103, 1.9295, 1.9487, 1.9679,
        1.9872, 2.0064, 2.0256, 2.0449, 2.0641, 2.0833, 2.1026, 2.1218, 2.1410,
        2.1603, 2.1795, 2.1987, 2.2179, 

In [219]:
count_data["count"] = round(best_period_count.tolist()[-1])
try:
    if (out_df.loc[(out_df['class'] == event_class) & (out_df['condition'] == event_user), 'count']):
        out_df.loc[(out_df['class'] == event_class) & (out_df['condition'] == event_user), 'count'] = count_data["count"]
except:
    out_df = pd.concat([out_df, pd.DataFrame([count_data])])

In [220]:
out_df.to_csv(out_csv_file, index=False)

In [221]:
out_df
# out_df.loc[(out_df['class'] == event_class) & (out_df['condition'] == event_user), 'count'] = count_data["count"]

,class,condition,count
0,class4,user02_led,3
1,class4,user02_led,3
2,class3,user02_led,11
3,class3,user02_lab,11
0,class3,user02_natural,3


In [120]:
import torch
import psutil

# GPU Memory Usage
gpu_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MB
print(f"GPU Memory Usage: {gpu_memory:.2f} MB")

# CPU RAM Usage
ram_usage = psutil.Process().memory_info().rss / (1024 * 1024)  # Convert to MB
print(f"CPU RAM Usage: {ram_usage:.2f} MB")


GPU Memory Usage: 523.25 MB
CPU RAM Usage: 1670.50 MB


In [121]:
# Generate plots and videos
print(f'Save plots and video with counts to {OUT_VISUALIZATIONS_DIR}...')
os.makedirs(OUT_VISUALIZATIONS_DIR, exist_ok=True)
dist = torch.cdist(best_embeddings, best_embeddings, p=2)**2
tsm_img = plots.plot_heatmap(dist.numpy(), log_scale=True)
pca_img = plots.plot_pca(best_embeddings.numpy())
cv2.imwrite(os.path.join(OUT_VISUALIZATIONS_DIR, 'tsm.png'), tsm_img)
cv2.imwrite(os.path.join(OUT_VISUALIZATIONS_DIR, 'pca.png'), pca_img)

Save plots and video with counts to ./visualization/...


True